In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

In [2]:
dataset = pd.read_csv("X_train1.csv")#Feature Engineered dataset
X_train, X_test, y_train, y_test = train_test_split(dataset.drop(columns=['SalePrice']), dataset['SalePrice'], test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1168, 83), (292, 83), (1168,), (292,))

In [3]:
# y_.info()

In [4]:
# y_train = dataset[['SalePrice']]
# y_train.info()
X_train.head()
X_train.info()
y_train.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1168 entries, 254 to 1126
Data columns (total 83 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               1168 non-null   int64  
 1   MSSubClass       1168 non-null   float64
 2   MSZoning         1168 non-null   float64
 3   LotFrontage      1168 non-null   float64
 4   LotArea          1168 non-null   float64
 5   Street           1168 non-null   float64
 6   Alley            1168 non-null   float64
 7   LotShape         1168 non-null   float64
 8   LandContour      1168 non-null   float64
 9   Utilities        1168 non-null   float64
 10  LotConfig        1168 non-null   float64
 11  LandSlope        1168 non-null   float64
 12  Neighborhood     1168 non-null   float64
 13  Condition1       1168 non-null   float64
 14  Condition2       1168 non-null   float64
 15  BldgType         1168 non-null   float64
 16  HouseStyle       1168 non-null   float64
 17  OverallQual      

In [5]:
# X_train = dataset.drop(['Id','SalePrice'], axis=1)
# Remove 'Id' column from the already split X_train
X_train = X_train.drop(columns=['Id'])

In [6]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

# Initialize the feature selection model using Lasso
feature_sel_model = SelectFromModel(Lasso(alpha=0.01, random_state=42))

# Fit the model on X_train and y_train
feature_sel_model.fit(X_train, y_train)

# Get the selected features
selected_features = X_train.columns[feature_sel_model.get_support()]

# Apply the same feature selection to X_test
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Print number of selected features
print(f"Total features before selection: {X_train.shape[1]}")
print(f"Total features after selection: {len(selected_features)}")

Total features before selection: 82
Total features after selection: 12


In [7]:
feature_sel_model.get_support()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
        True,  True, False, False, False, False, False,  True, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False,  True, False, False, False, False,
        True, False, False,  True, False, False, False, False, False,
       False,  True, False, False, False, False,  True, False, False,
       False, False,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [8]:
# # # let's print the number of total and selected features

# # # this is how we can make a list of the selected features
# # selected_feat = X_train.columns[(feature_sel_model.get_support())]

# # # let's print some stats
# # print('total features: {}'.format((X_train.shape[1])))
# # print('selected features: {}'.format(len(selected_feat)))
# # print('features with coefficients shrank to zero: {}'.format(
# #     np.sum(feature_sel_model.estimator_.coef_ == 0)))


# # Apply feature selection
# feature_sel_model = SelectFromModel(Lasso(alpha=0.005, random_state=42))
# feature_sel_model.fit(X_train, y_train)

# # Extract selected features
# selected_feat = X_train.columns[feature_sel_model.get_support()]

# # Apply feature selection to X_test
# X_train_selected = X_train[selected_feat]
# X_test_selected = X_test[selected_feat]

# # Print feature selection stats
# print('Total features before selection:', X_train.shape[1])
# print('Selected features:', len(selected_feat))
# print('Features with coefficients shrank to zero:', np.sum(feature_sel_model.estimator_.coef_ == 0))

In [9]:
# selected_feat


In [10]:
X_train=X_train[selected_features]

In [11]:
selected_features

Index(['OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'CentralAir', 'GrLivArea', 'FullBath', 'Fireplaces', 'GarageCars',
       'WoodDeckSF', 'OpenPorchSF'],
      dtype='object')

In [12]:
y_train.head()

254     11.884489
1066    12.089539
638     11.350407
799     12.072541
380     11.751942
Name: SalePrice, dtype: float64

In [13]:
from sklearn.linear_model import LinearRegression
import joblib

In [14]:
model = LinearRegression()
model.fit(X_train_selected, y_train)
print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)

Intercept: 10.643876398907027
Coefficients: [ 0.8268959  -0.18214685 -0.13195094 -0.02513471  0.15990865  0.10076548
  1.14258727 -0.02674805  0.16542853  0.36275267  0.03721724  0.0324454 ]


In [15]:
joblib.dump(model, "linear_regression_model.pkl")  # Save model as a .pkl file

['linear_regression_model.pkl']

In [16]:
model_loaded = joblib.load("linear_regression_model.pkl")  # Load trained model

In [17]:
# Assume 'new_data' is a DataFrame with the same feature columns as X_train
predicted_price = model_loaded.predict(X_test_selected)

# import numpy as np

# Apply inverse log transformation
predicted_price_original = np.exp(predicted_price)
print(f"Predicted SalePrice : {predicted_price}")
print(f"Predicted SalePrice (Original Scale): {predicted_price_original}")

Predicted SalePrice : [11.89445091 12.63559282 11.60642459 12.10631521 12.61861209 11.30948045
 12.35788879 12.08388147 11.26872641 11.66597954 11.86246124 11.62456408
 11.43295911 12.11090695 12.15081173 11.75375439 12.26351474 11.78788823
 11.6183508  12.25600429 12.06497275 12.24890138 12.14911189 11.67049244
 12.20194535 12.01295243 12.17786103 11.50933276 12.09176446 12.16079008
 11.73496238 12.52928554 12.08206813 11.49602456 12.46813229 11.89833037
 11.90576237 12.30600886 12.68799373 11.47473839 11.80310121 12.46922688
 11.65397349 12.4791371  11.72512227 11.81288181 11.61061385 11.6500538
 12.87539102 11.78491366 11.62426375 12.22841525 11.78566183 12.55624859
 11.97414428 12.35035954 12.335605   11.8639663  11.99383891 11.58466294
 10.99834745 11.81512184 12.57490073 12.36777115 12.48779028 12.26554026
 11.50990878 12.68796642 11.65103431 12.14986752 11.84023227 11.80756828
 11.48341183 11.30747906 12.81879012 12.15205972 12.63716716 12.56290435
 11.67929425 11.60688347 11.82

In [18]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Apply inverse log transformation on predictions and actual values
# y_test_original = np.exp(y_test)  # Convert test targets back to original scale
# predicted_price_original = np.exp(predicted_price)  # Convert predictions back to original scale

# Calculate performance metrics
mae = mean_absolute_error(y_test, predicted_price)
mse = mean_squared_error(y_test, predicted_price)
rmse = np.sqrt(mse)  # Root Mean Squared Error
r2 = r2_score(y_test, predicted_price)

# Print results
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.4f}")

Mean Absolute Error (MAE): 0.13
Mean Squared Error (MSE): 0.03
Root Mean Squared Error (RMSE): 0.17
R² Score: 0.8517
